#### WandB 사용해보기

##### 시작 전
- pip install wandb

- wandb login  (입력 후 나오는 url로 접속하여 본인의 api를 terminal에 붙여넣기) 

#### 프로젝트 생성

In [1]:
import wandb

In [2]:
# config setting
config = {
    'epochs': 10,
    'num_classes': 10,
    'batch_size': 128,
    'conv_filters': [128, 64, 64, 32],  # 각 conv layer의 필터 수
    'kernel_size': 3,               # 모든 conv layer에 동일한 3x3 커널 사용
    'pooling': {
        'kernel_size': 2,
        'stride': 2
    },
    'fc_layers': [2048, 512],    # classifier의 fully-connected layer 구조
    'dropout_rate': 0.1,
    'weight_decay': 0.0005,
    'learning_rate': 0.001,
    'dataset': 'CIFAR10',
    'architecture': 'CNN',  # 모델 이름 또는 구조 설명
    'seed': 42
}

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm

wandb.init(project="first-practice", config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dataset setting
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

c:\Users\islab\.conda\envs\mings\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kmings (kming). Use `wandb login --relogin` to force relogin


Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Teacher(nn.Module):
    def __init__(self, config):
        super(Teacher, self).__init__()
        self.config = config

        # config 값 할당
        conv_filters = config['conv_filters']      
        kernel_size = config['kernel_size']          
        pool_kernel = config['pooling']['kernel_size']  
        pool_stride = config['pooling']['stride']      
        fc_layers = config['fc_layers']
        num_classes = config['num_classes']

        self.features = nn.Sequential(
            nn.Conv2d(3, conv_filters[0], kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(conv_filters[0], conv_filters[1], kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_kernel, stride=pool_stride),
            nn.Conv2d(conv_filters[1], conv_filters[2], kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(conv_filters[2], conv_filters[3], kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_kernel, stride=pool_stride),
        )
        self.classifier = nn.Sequential(
            nn.Linear(fc_layers[0], fc_layers[1]),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(fc_layers[1], num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    

In [5]:
def train(model, train_loader, criterion, optimizer, config):
    for epoch in tqdm(range(config.epochs)):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).to(device)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = running_loss / len(train_loader)
        wandb.log({"loss": avg_loss})
        print(f"TRAIN: EPOCH {epoch + 1:04d} / {config.epochs:04d} | Epoch LOSS {avg_loss:.4f}")

def test(model, test_loader, config):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    wandb.log({"Test_accuracy": accuracy})
    return accuracy

In [6]:
def run(config=None):
    config = wandb.config

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

    model = Teacher(config).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    train(model, train_loader, criterion, optimizer, config)
    test(model, test_loader, config)

    return model

In [7]:
model = run()

 10%|█         | 1/10 [00:08<01:19,  8.79s/it]

TRAIN: EPOCH 0001 / 0010 | Epoch LOSS 1.3935


 20%|██        | 2/10 [00:16<01:05,  8.17s/it]

TRAIN: EPOCH 0002 / 0010 | Epoch LOSS 0.9187


 30%|███       | 3/10 [00:24<00:55,  7.94s/it]

...

 90%|█████████ | 9/10 [01:09<00:07,  7.62s/it]

TRAIN: EPOCH 0009 / 0010 | Epoch LOSS 0.1577


100%|██████████| 10/10 [01:17<00:00,  7.76s/it]

TRAIN: EPOCH 0010 / 0010 | Epoch LOSS 0.1276


Test Accuracy: 75.83%
